# ICR-pycaret

In [1]:
!pip install lightgbm
!pip install catboost
!pip install xgboost


[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
 !pip install pycaret==3.0.2


[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
from pycaret.classification import *
import pycaret

In [2]:
print(pycaret.__version__)

3.0.2


## imports

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer

## read files

In [4]:
df = pd.read_csv("../input/icr-identify-age-related-conditions/train.csv", index_col=[0])
df.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
Id,,,,,,,,,,,,,,,,,,,,,
000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [5]:
df_test = pd.read_csv("../input/icr-identify-age-related-conditions/test.csv", index_col=[0])
df_test

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
Id,,,,,,,,,,,,,,,,,,,,,
00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df['EJ'] = df['EJ'].map({'A': 0.0, 'B': 1.0}, na_action=None)
df_test['EJ'] = df_test['EJ'].map({'A': 0.0, 'B': 1.0}, na_action=None)

## pycaret

In [7]:
def balanced_log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    balanced_log_loss_score = (-w0/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(1-y_pred))) - w1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred)))) / (w0+w1)
    return balanced_log_loss_score

# 独自の評価関数をスコアリング関数に変換
balanced_log_loss_scorer = make_scorer(balanced_log_loss, greater_is_better=False)

In [8]:
exp1 = setup(df, target = 'Class', ignore_features = ['Id'], session_id=42)

,Description,Value
0,Session id,42
1,Target,Class
2,Target type,Binary
3,Original data shape,"(617, 57)"
4,Transformed data shape,"(617, 57)"
5,Transformed train set shape,"(431, 57)"
6,Transformed test set shape,"(186, 57)"
7,Ignore features,1
8,Numeric features,56
9,Rows with missing values,11.2%


In [9]:
count_1 = df['Class'].sum()
count_0 = df['Class'].count() - count_1

# Calculate the ratio
ratio = count_1 / count_0
ratio

0.21218074656188604

In [66]:
# モデルの構築
models = compare_models(n_select = 3, probability_threshold = ratio)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9350,0.9682,0.8393,0.8119,0.8165,0.7775,0.7838,0.0370
xgboost,Extreme Gradient Boosting,0.9258,0.9592,0.8429,0.7696,0.7993,0.7542,0.7592,0.0410
catboost,CatBoost Classifier,0.9142,0.9653,0.8429,0.7236,0.7724,0.7204,0.7282,0.0340
gbc,Gradient Boosting Classifier,0.8957,0.9412,0.7179,0.6988,0.6982,0.6361,0.6430,0.0330
ridge,Ridge Classifier,0.8865,0.0000,0.5393,0.7729,0.6235,0.5599,0.5789,0.0250
rf,Random Forest Classifier,0.8726,0.9436,0.8929,0.5947,0.7102,0.6336,0.6578,0.0410
et,Extra Trees Classifier,0.8701,0.9458,0.9071,0.5843,0.7082,0.6302,0.6573,0.0530
lda,Linear Discriminant Analysis,0.8634,0.8846,0.6857,0.6382,0.6418,0.5599,0.5731,0.0280
qda,Quadratic Discriminant Analysis,0.8516,0.6635,0.2339,0.6167,0.3301,0.2824,0.3275,0.0290
dt,Decision Tree Classifier,0.8448,0.7208,0.5286,0.5468,0.5309,0.4392,0.4434,0.0290


In [67]:
for i in range(len(models)):
    models[i] = tune_model(models[i], optimize = "F1", fold = 10, n_iter = 25)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9091,0.9549,0.7500,0.7500,0.7500,0.6944,0.6944
1,0.9302,0.9643,0.8571,0.7500,0.8000,0.7580,0.7605
2,0.9535,0.9921,1.0000,0.7778,0.8750,0.8470,0.8571
3,0.9302,0.9722,0.8571,0.7500,0.8000,0.7580,0.7605
4,0.8605,0.9127,0.7143,0.5556,0.6250,0.5409,0.5474
5,0.9767,0.9921,0.8571,1.0000,0.9231,0.9095,0.9132
6,0.9535,0.9929,0.8750,0.8750,0.8750,0.8464,0.8464
7,0.9535,0.9857,0.8750,0.8750,0.8750,0.8464,0.8464
8,0.9302,0.9429,0.8750,0.7778,0.8235,0.7802,0.7823


Fitting 10 folds for each of 25 candidates, totalling 250 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9318,0.9167,0.7500,0.8571,0.8000,0.7591,0.7616
1,0.8837,0.9841,1.0000,0.5833,0.7368,0.6687,0.7087
2,0.9070,0.9802,1.0000,0.6364,0.7778,0.7226,0.7521
3,0.8837,0.9683,1.0000,0.5833,0.7368,0.6687,0.7087
4,0.8605,0.8929,0.7143,0.5556,0.6250,0.5409,0.5474
5,0.9767,0.9841,0.8571,1.0000,0.9231,0.9095,0.9132
6,0.9767,0.9893,1.0000,0.8889,0.9412,0.9267,0.9292
7,0.9535,0.9857,0.8750,0.8750,0.8750,0.8464,0.8464
8,0.9302,0.9750,0.8750,0.7778,0.8235,0.7802,0.7823


Fitting 10 folds for each of 25 candidates, totalling 250 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9091,0.8750,0.7500,0.7500,0.7500,0.6944,0.6944
1,0.8837,0.9524,0.8571,0.6000,0.7059,0.6362,0.6519
2,0.9535,1.0000,1.0000,0.7778,0.8750,0.8470,0.8571
3,0.9070,0.9524,0.8571,0.6667,0.7500,0.6940,0.7022
4,0.9302,0.8651,0.7143,0.8333,0.7692,0.7284,0.7314
5,0.9535,0.9325,0.8571,0.8571,0.8571,0.8294,0.8294
6,0.9767,1.0000,0.8750,1.0000,0.9333,0.9193,0.9223
7,0.8605,0.9286,0.7500,0.6000,0.6667,0.5798,0.5856
8,0.9070,0.9536,0.7500,0.7500,0.7500,0.6929,0.6929


Fitting 10 folds for each of 25 candidates, totalling 250 fits


In [38]:
# rf = create_model('rf')
# tuned_rf = tune_model(rf, n_iter = 10, optimize = 'r2')

In [68]:
blender_specific = blend_models(estimator_list = models)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9318,0.9306,0.6250,1.0000,0.7692,0.7317,0.7596
1,0.9302,0.9722,0.8571,0.7500,0.8000,0.7580,0.7605
2,0.9535,0.9960,1.0000,0.7778,0.8750,0.8470,0.8571
3,0.9070,0.9762,0.8571,0.6667,0.7500,0.6940,0.7022
4,0.9302,0.8929,0.7143,0.8333,0.7692,0.7284,0.7314
5,0.9535,0.9921,0.7143,1.0000,0.8333,0.8072,0.8226
6,0.9535,0.9964,0.8750,0.8750,0.8750,0.8464,0.8464
7,0.9302,0.9750,0.6250,1.0000,0.7692,0.7307,0.7587
8,0.9070,0.9643,0.7500,0.7500,0.7500,0.6929,0.6929


In [69]:
# tuned_blender = tune_model(blender_specific) #, optimize = "mae", fold = 10, n_iter = 50)
# tuned_blender
predict_model(blender_specific, data=df[df.columns[:-1]], raw_score=True)

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FS,GB,GE,GF,GH,GI,GL,prediction_label,prediction_score_0,prediction_score_1
Id,,,,,,,,,,,,,,,,,,,,,
000ff2bfdfe9,0.209377,3109.033203,85.200150,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.587402,...,0.094822,11.339138,72.611061,2003.810303,22.136229,69.834946,0.120343,1,0.0119,0.9881
007255e47698,0.145282,978.764160,85.200150,36.968887,8.138688,3.632190,0.025578,13.517790,1.229900,5496.928223,...,0.568932,9.292698,72.611061,27981.562500,29.135429,32.131996,21.978001,0,0.9987,0.0013
013f2bd269f5,0.470030,2635.106445,85.200150,32.360554,8.138688,6.732840,0.025578,12.824570,1.229900,5135.780273,...,1.198821,37.077770,88.609436,13676.958008,28.022852,35.192677,0.196941,0,0.9191,0.0809
043ac50845d5,0.252107,3819.651855,120.201614,77.112206,8.138688,3.685344,0.025578,11.053708,1.229900,4169.677246,...,0.284466,18.529585,82.416801,2094.262451,39.948654,90.493248,0.155829,0,0.9858,0.0142
044fb8a146ec,0.380297,3733.048340,85.200150,14.103738,8.138688,3.942255,0.054810,3.396778,102.151978,5728.733887,...,0.121914,16.408728,146.109940,8524.370117,45.381317,36.262627,0.096614,0,0.7588,0.2412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fd3dafe738fd,0.149555,3130.059570,123.763596,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,4157.684570,...,0.067730,8.967128,217.148560,8095.932617,24.640463,69.191940,21.978001,0,0.9939,0.0061
fd895603f071,0.435846,5462.034180,85.200150,46.551006,15.973224,5.979825,0.025882,12.622906,3.777550,5654.075684,...,0.426699,35.896420,496.994202,3085.308105,29.648928,124.808868,0.145340,0,0.9696,0.0304
fd8ef6377f76,0.427300,2459.107178,130.138580,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,5888.877930,...,0.067730,19.962091,128.896896,6474.652832,26.166073,119.559418,21.978001,0,0.9994,0.0006


In [82]:
pred = predict_model(blender_specific, data=df[df.columns[:-1]], raw_score=True)[['prediction_score_1']].clip(0, 1)
print(pred)

              prediction_score_1
Id                              
000ff2bfdfe9              0.9881
007255e47698              0.0013
013f2bd269f5              0.0809
043ac50845d5              0.0142
044fb8a146ec              0.2412
...                          ...
fd3dafe738fd              0.0061
fd895603f071              0.0304
fd8ef6377f76              0.0006
fe1942975e40              0.0054
ffcca4ded3bb              0.0050

[617 rows x 1 columns]


In [83]:
pred['Class'] = df['Class']

In [84]:
pred

,prediction_score_1,Class
Id,,
000ff2bfdfe9,0.9881,1
007255e47698,0.0013,0
013f2bd269f5,0.0809,0
043ac50845d5,0.0142,0
044fb8a146ec,0.2412,1
...,...,...
fd3dafe738fd,0.0061,0
fd895603f071,0.0304,0
fd8ef6377f76,0.0006,0


In [85]:
balanced_log_loss(pred['Class'], pred['prediction_score_1'])

0.14894542113099335

In [88]:
evaluate_model(blender_specific)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [74]:
save_model(blender_specific, 'blend_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['AB', 'AF', 'AH', 'AM', 'AR', 'AX',
                                              'AY', 'AZ', 'BC', 'BD ', 'BN',
                                              'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC',
                                              'CD ', 'CF', 'CH', 'CL', 'CR',
                                              'CS', 'CU', 'CW ', 'DA', 'DE',
                                              'DF', 'DH', 'DI', ...],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_...
                                                              max_depth=8,
         

# inference

In [75]:
model = load_model('blend_model')

Transformation Pipeline and Model Successfully Loaded


In [76]:
pred = predict_model(model, data=df_test, raw_score=True)['prediction_score_1'].to_numpy()
pred

array([0.2983, 0.2983, 0.2983, 0.2983, 0.2983])

In [77]:
sub = pd.read_csv('../input/icr-identify-age-related-conditions/sample_submission.csv')
sub['class_1'] = pred
sub['class_0'] = 1 - pred
sub.to_csv('submission.csv', index=False)
sub

,Id,class_0,class_1
0,00eed32682bb,0.7017,0.2983
1,010ebe33f668,0.7017,0.2983
2,02fa521e1838,0.7017,0.2983
3,040e15f562a2,0.7017,0.2983
4,046e85c7cc7f,0.7017,0.2983
